In [1]:
public double ErroQuadraticoMedio(int _n_linhas, int _n_colunas,int _n_neuronios, double[,] _x, double[] _d, double[] _w ,double _thetaSaida,double[] _thetaEscondida,double[,] _v){
    
    double somaQuadradoDaDiferenca = 0;

    int c = 1;
    Console.WriteLine($"=====================================");
    Console.WriteLine($"==========Iniciando cálculo do Erro Quadrático=======");
    Console.WriteLine($"=====================================");
    for(int linha=0;linha<_n_linhas;linha++){
                

        Console.WriteLine($"Entrada:{linha+c}");
        
        // Calculando z
        double[] _z = new double[_n_neuronios];

        for(int j=0;j<_n_neuronios;j++){
            
            double zSoma = 0;

            for(int i=0;i<_n_colunas;i++){
                zSoma+=_v[i,j]*_x[linha,i];
                Console.WriteLine($"_v_{i}_{j}:{_v[i,j]} ,_x_{linha}_{j}:{_x[linha,i]} ");
            }
            _z[j] = zSoma + _thetaEscondida[j];
            _z[j] = Math.Tanh(_z[j]);
            Console.WriteLine($"_z_{j+c}:{_z[j]}");
        }

        double y =0;
        double yEstrela=0;
        //calculando somatório das variáveis da entrada
        for(int j=0;j<_n_neuronios;j++){

            yEstrela += _z[j]*_w[j];

        }

        //adicionando theta ao somatório
        yEstrela = yEstrela + _thetaSaida;
        Console.WriteLine($"yEstrela:{yEstrela}");

        y=Math.Tanh(yEstrela);
        
        Console.WriteLine($"y_:{y}");
        var quadradoDiferenca = Math.Pow((_d[linha]-y),2);
        Console.WriteLine($"quadradoDiferenca:{quadradoDiferenca}");
        somaQuadradoDaDiferenca+= quadradoDiferenca;

    }

    Console.WriteLine($"ErroQuadradoMedio:{somaQuadradoDaDiferenca/2}");
    return somaQuadradoDaDiferenca/2;

}
 
public (double[] _w, double _thetaSaida, double[] _thetaEscondida,double[,] _v) MLPMomento(int n_iteracoes,int n_neuronios,double[] thetaEscondida, int[][] ordem, double[] w, double alpha, double[,] x, int n_linhas, int n_colunas,double[] d,double erroAceito,double thetaSaida, double[,] v, double[] z, double taxaMomento){
    
    int contagem_iteracoes = 1;
    bool continuarLoop = true;
    int c = 1;
    double[] w_anterior = new double[n_neuronios];
    // número máximo de iterações
    while(contagem_iteracoes<=n_iteracoes && continuarLoop){
        
        Console.WriteLine($"=====================================================");
        Console.WriteLine($"================   iteracao{contagem_iteracoes}  =============");
        Console.WriteLine($"=====================================================");

        //calculando z
        foreach(var linha in ordem[contagem_iteracoes-1]){
            Console.WriteLine($"=====================================================");
            Console.WriteLine($"================   Entrada_{linha}  =============");
            Console.WriteLine($"=====================================================");
            for(int j=0;j<n_neuronios;j++){
                
                double _z = 0;

                for(int i = 0;i<n_colunas;i++){
                    Console.WriteLine($"v_{i+c}_{j+c}:{v[i,j]} , x_{linha}_{i+c}:{x[linha-c,i]}");
                    _z+=v[i,j]*x[linha-c,i];
                    Console.WriteLine($"_z_{i+c}:{_z}");
                }
                _z = _z + thetaEscondida[j];
                Console.WriteLine($"z*_{j+c}:{_z}");
                _z = Math.Tanh(_z);
                Console.WriteLine($"z_{j+c}:{_z}");
                z[j]=_z;

            }

            //calculando y
            double y=0;
            for(int j=0;j<n_neuronios;j++){
                y += w[j]*z[j];
            }

            y= y+thetaSaida;

            y=Math.Tanh(y);

            Console.WriteLine($"y:{y}");

            //Calculando w
            
            if(linha==1 && contagem_iteracoes==1){
                for(int j=0;j<n_neuronios;j++){

                    w[j] = alpha*(1-Math.Pow(y,2))*(d[linha-c]-y)*z[j] + w[j] ;
                    Console.WriteLine($"w_{j+c}:{w[j]}");

                }    
            }else{
                for(int j=0;j<n_neuronios;j++){

                    w[j] = alpha*(1-Math.Pow(y,2))*(d[linha-c]-y)*z[j] + w[j] + taxaMomento*(w[j]-w_anterior[j]);
                    Console.WriteLine($"w_{j+c}:{w[j]}");
                    w_anterior[j] = w[j];
                }    
            }
            
            

            //calculando theta
            thetaSaida = alpha*(1-Math.Pow(y,2))*(d[linha-c]-y) + thetaSaida;
            Console.WriteLine($"thetaSaida:{thetaSaida}");

            for(int i =0;i<n_colunas;i++){
                for(int j = 0;j<n_neuronios;j++){
                    //v[i,j]= alpha*y*(1-y)*(d[linha-c]-y)*w[j]*z[j]*(1-z[j])*x[linha-c,i]+v[i,j];
                    v[i,j]= alpha*(d[linha-c]-y)*(1-Math.Pow(y,2))*w[j]*(1-Math.Pow(z[j],2))*x[linha-c,i] +v[i,j];
                    Console.WriteLine($"v_{i+c}_{j+c}:{v[i,j]}");
                }
            }

            for(int j = 0;j<n_neuronios;j++){
                //thetaEscondida[j]= alpha*y*(1-y)*(d[linha-c]-y)*w[j]*z[j]*(1-z[j])+thetaEscondida[j];
                thetaEscondida[j]= alpha*(d[linha-c]-y)*(1-Math.Pow(y,2))*w[j]*(1-Math.Pow(z[j],2))+thetaEscondida[j];
                Console.WriteLine($"thetaEscondida_{j+c}:{thetaEscondida[j]}");
            }
        }

        continuarLoop = !(ErroQuadraticoMedio(n_linhas, n_colunas,n_neuronios, x, d, w ,thetaSaida,thetaEscondida,v) <= erroAceito);
        alpha = alpha*0.95;
        contagem_iteracoes++;
        
    }

    return (w, thetaSaida, thetaEscondida,v);
}

In [2]:
int c = 1;
int n_linhas = 13;
int n_colunas = 1;
int n_neuronios = 3;
double[,] x = new double[n_linhas,n_colunas];
x[1-c,1-c]=-3;
x[2-c,1-c]=-2.5;
x[3-c,1-c]=-2;
x[4-c,1-c]=-1.5;
x[5-c,1-c]=-1;
x[6-c,1-c]=-0.5;
x[7-c,1-c]=0;
x[8-c,1-c]=0.5;
x[9-c,1-c]=1;
x[10-c,1-c]=1.5;
x[11-c,1-c]=2;
x[12-c,1-c]=2.5;
x[12-c,1-c]=3;

//iniciando vetor da variável resposta
double[] d = new double[n_linhas];
d[1-c]=-10;
d[2-c]=-6;
d[3-c]=-3.6;
d[4-c]=-2.1;
d[5-c]=-1.2;
d[6-c]=-0.5;
d[7-c]=0;
d[8-c]=0.52;
d[9-c]=1.18;
d[10-c]=2;
d[11-c]=3.6;
d[12-c]=6.05;
d[13-c]=10.02;


//delta alpha e theta iniciais
double alpha = 0.4;
double[] thetaEscondida = new double[n_neuronios];
thetaEscondida[1-c]=0.5;
thetaEscondida[2-c]=0.5;
thetaEscondida[3-c]=0.5;
double thetaSaida =0.5;
double taxaMomento = 0.7;

//iniciando o vetor de pesos 
double[] w = new double[n_neuronios];
w[1-c] = 0.3;
w[2-c] = 0.3;
w[3-c] = 0.3;

double[,] v = new double[n_colunas,n_neuronios];
v[1-c,1-c] = 0.3 ;
v[1-c,2-c] = 0.3 ;
v[1-c,3-c] = 0.3 ;

double[] z = new double[n_neuronios];

//lista contendo a ordem específica das entradas em cada iteração, usada para "embaralhar" e diversificar o treinamento
int n_iteracoes = 10;
int[][] ordem = new int[n_iteracoes][];
ordem[1-c] = [1,2,3,4,5,6,7,8,9,10,11,12,13];
ordem[2-c] = [1,2,3,4,5,6,7,8,9,10,11,12,13];
ordem[3-c] = [1,2,3,4,5,6,7,8,9,10,11,12,13];
ordem[4-c] = [1,2,3,4,5,6,7,8,9,10,11,12,13];
ordem[5-c] = [1,2,3,4,5,6,7,8,9,10,11,12,13];
ordem[6-c] = [1,2,3,4,5,6,7,8,9,10,11,12,13];
ordem[7-c] = [1,2,3,4,5,6,7,8,9,10,11,12,13];
ordem[8-c] = [1,2,3,4,5,6,7,8,9,10,11,12,13];
ordem[9-c] = [1,2,3,4,5,6,7,8,9,10,11,12,13];
ordem[10-c] = [1,2,3,4,5,6,7,8,9,10,11,12,13];

In [3]:
public double[] NormalizaVetorResposta(double[] d){
    double maximo = d.Max();
    double minimo = d.Min();
    for(int i = 0;i<d.Length;i++){
        d[i]= 2*((d[i]-minimo)/(maximo-minimo))-1;
        Console.WriteLine($"d_{i+c}:{d[i]}");
    }
    return d;
}

public double[,] NormalizaEntrada(double[,] x , int n_linhas, int n_colunas){
    
    for(int j = 0;j<n_colunas;j++){

        double maximo = 0;
        double minimo = double.MaxValue;
        
        for(int i = 0;i<n_linhas;i++){
            if(x[i,j]>maximo){
                maximo=x[i,j];
            }

            if(x[i,j]<minimo){
                minimo = x[i,j];
            }

        }

        for(int i = 0;i<n_linhas;i++){

            x[i,j]= 2*((x[i,j]-minimo)/(maximo-minimo))-1;
            Console.WriteLine($"x_{i+c}_{j+c}:{x[i,j]}");

        }
    }
    return x;
}

In [4]:
d = NormalizaVetorResposta(d);
x = NormalizaEntrada(x,n_linhas,n_colunas);

d_1:-1
d_2:-0,6003996003996004
d_3:-0,36063936063936064
d_4:-0,21078921078921076
d_5:-0,12087912087912078
d_6:-0,05094905094905089
d_7:-0,0009990009990009652
d_8:0,05094905094905089
d_9:0,11688311688311681
d_10:0,19880119880119884
d_11:0,3586413586413586
d_12:0,6033966033966034
d_13:1
x_1_1:-1
x_2_1:-0,8333333333333334
x_3_1:-0,6666666666666667
x_4_1:-0,5
x_5_1:-0,33333333333333337
x_6_1:-0,16666666666666663
x_7_1:0
x_8_1:0,16666666666666674
x_9_1:0,33333333333333326
x_10_1:0,5
x_11_1:0,6666666666666667
x_12_1:1
x_13_1:0


In [5]:
var erroAceito = 0.5;
//realização do treimanto passando os dados de entrada
(double[] _w, double _thetaSaida, double[] _thetaEscondida,double[,] _v)= MLPMomento(n_iteracoes, n_neuronios, thetaEscondida,  ordem,  w,  alpha,  x,  n_linhas,  n_colunas, d, erroAceito, thetaSaida,  v,  z,taxaMomento);

//plotando pesos w e o theta do resultado do treinamento
var resultado = "";
for(int i =0;i<_w.Length;i++){
    resultado += $"w_{i+c}:{w[i]} | ";
}
resultado +=$"theta:{_thetaSaida} | ";

Console.WriteLine($"Resultado: {resultado}");

================   iteracao1  =============
================   Entrada_1  =============
v_1_1:0,3 , x_1_1:-1
_z_1:-0,3
z*_1:0,2
z_1:0,197375320224904
v_1_2:0,3 , x_1_1:-1
_z_1:-0,3
z*_2:0,2
z_2:0,197375320224904
v_1_3:0,3 , x_1_1:-1
_z_1:-0,3
z*_3:0,2
z_3:0,197375320224904
y:0,5899815642501437
w_1:0,21816475238185065
w_2:0,21816475238185065
w_3:0,21816475238185065
thetaSaida:0,0853825720211614
v_1_1:0,38693105509665066
v_1_2:0,38693105509665066
v_1_3:0,38693105509665066
thetaEscondida_1:0,41306894490334933
thetaEscondida_2:0,41306894490334933
thetaEscondida_3:0,41306894490334933
================   Entrada_2  =============
v_1_1:0,38693105509665066 , x_2_1:-0,8333333333333334
_z_1:-0,32244254591387556
z*_1:0,09062639898947378
z_1:0,09037910216395048
v_1_2:0,38693105509665066 , x_2_1:-0,8333333333333334
_z_1:-0,32244254591387556
z*_2:0,09062639898947378
z_2:0,09037910216395048
v_1_3:0,38693105509665066 , x_2_1:-0,8333333333333334
_z_1:-0,32244254591387556
z*_3:0,09062639898947378
z_3:0,0